In [1]:
!nvidia-smi

Mon Apr 29 07:13:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   52C    P8              17W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install opencv-python
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.5 MB/s eta 0:00:00


In [3]:
!pip install -e .

Obtaining file:///home/jupyter/unattended-baggage-yolov8-modification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.2-0.editable-py3-none-any.whl size=16444 sha256=de9344dcbcd62f5e19643da047a29eea98931b9c4e9b87117e4dd6fd046784a3
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-_tfgt8xo/wheels/16/24/4e/77eb3faafdc0df8f4050b5c10cf91dc4af4801edeb0d66c251
Successfully built ultralytics


In [10]:
from ultralytics import YOLO
# Load a model
model = YOLO('ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml')
model.load('yolov8n.pt') # loading pretrain weights

# Train the model
results = model.train(data='coco8.yaml', epochs=2, imgsz=640, batch=4, device=[0, 1])

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Transferred 210/361 items from pretrained weights
New https://pypi.org/project/ultralytics/8.2.4 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml, data=coco8.yaml, epochs=2, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
val: Scanning /home/jupyter/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 67 weight(decay=0.0005), 66 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          4         17          0          0          0          0

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2      1.43G      3.394      5.425      4.188         12        640: 100%|██████████| 1/1 [00:00<00:00,  6.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 13.39it/s]


                   all          4         17          0          0          0          0

2 epochs completed in 0.001 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6/weights/last.pt, 6.6MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6/weights/best.pt, 6.6MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
YOLOv8-BiLevelRoutingAttention summary (fused): 179 layers, 3169184 parameters, 0 gradients, 9.4 GFLOPs
                   all          4         17          0          0          0          0


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 19.05it/s]


Speed: 0.2ms preprocess, 11.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/train6
